This notebook identifies a list of all combination ICS inhalers that have not been categorised as high dose i.e. low and Medium dose inhalers. This is a pragmatic approach to work out a steroid load for a patient without using dose syntax.

- [All ICS inhalers](#ai)
- [low and medium dose ICS inhalers](#lm)

In [1]:
#import libraries
from ebmdatalab import bq
import os
import pandas as pd

## All ICS Combination Agent Inhalers <a id='ai'></a>

In [17]:


sql = '''
WITH bnf_codes AS (  
  SELECT DISTINCT bnf_code FROM measures.dmd_objs_with_form_route WHERE 
  (bnf_code LIKE '0302000C0%' OR #BNF Beclometasone dipropionate
  bnf_code LIKE '0301011AB%'  OR #BNF BeclometDiprop/Formoterol/Glycopyrronium",
  bnf_code LIKE '0302000K0%'  OR #BNF budesonide
  bnf_code LIKE '0302000U0%'  OR #BNF Ciclesonide
  bnf_code LIKE '0302000V0%'  OR #BNF Fluticasone furoate 
  bnf_code LIKE '0302000N0%'  OR #BNF Fluticasone propionate 
  bnf_code LIKE '0302000R0%')   #BNF Mometasone Furoate
  AND
  (form_route LIKE '%pressurizedinhalation.inhalation' OR form_route LIKE 'powderinhalation.inhalation%')
   )

SELECT "vmp" AS type, vmp.id as id, vmp.bnf_code as bnf_code, vmp.nm as nm
FROM dmd.vmp as vmp
INNER JOIN dmd.vpi as vpi
ON
vmp.id = vpi.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)
GROUP BY vmp.id, vmp.bnf_code, vmp.nm
HAVING COUNT(ing) > 1

UNION ALL

SELECT "amp" AS type, amp.id as id, amp.bnf_code as bnf_code, amp.descr as nm
FROM dmd.amp as amp
INNER JOIN dmd.vpi as vpi
ON
amp.vmp = vpi.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)
GROUP BY amp.id, amp.bnf_code, amp.descr
HAVING COUNT(ing) > 1

ORDER BY type, bnf_code, id'''

all_inhaler_ics_combi_agent = bq.cached_read(sql, csv_path=os.path.join('..','data','all_inhaler_ics_combi_agent.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
all_inhaler_ics_combi_agent.info()

Downloading: 100%|██████████| 69/69 [00:00<00:00, 461.06rows/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      69 non-null     object
 1   id        69 non-null     int64 
 2   bnf_code  69 non-null     object
 3   nm        69 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.3+ KB


In [18]:
#import csv from other notebook dealing with high dose
dose_high_ics_combi_agent = pd.read_csv('../data/highdose_inhaledsteroid_combiagent_codelist.csv')
dose_high_ics_combi_agent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      26 non-null     object
 1   id        26 non-null     int64 
 2   bnf_code  26 non-null     object
 3   nm        26 non-null     object
dtypes: int64(1), object(3)
memory usage: 960.0+ bytes


In [19]:
## here we merge and create an indicator to see which ones are in both
combine = pd.merge(all_inhaler_ics_combi_agent,dose_high_ics_combi_agent, how='outer', indicator=True)
combine

,type,id,bnf_code,nm,_merge
0,amp,34681611000001100,0301011ABBBAAA0,Trimbow 87micrograms/dose / 5micrograms/dose / 9micrograms/dose inhaler (Chiesi Ltd),left_only
1,amp,3292811000001106,0302000C0BDAAAN,Ventide inhaler (GlaxoSmithKline UK Ltd),left_only
2,amp,12906411000001100,0302000C0BQAABX,Fostair 100micrograms/dose / 6micrograms/dose inhaler (Chiesi Ltd),left_only
3,amp,31063411000001101,0302000C0BQABBZ,Fostair 200micrograms/dose / 6micrograms/dose inhaler (Chiesi Ltd),both
4,amp,26112111000001106,0302000C0BRAABY,Fostair NEXThaler 100micrograms/dose / 6micrograms/dose dry powder inhaler (Chiesi Ltd),left_only
5,amp,31063111000001106,0302000C0BRABCA,Fostair NEXThaler 200micrograms/dose / 6micrograms/dose dry powder inhaler (Chiesi Ltd),both
6,amp,3294211000001101,0302000K0BDAAAL,Symbicort 100/6 Turbohaler (AstraZeneca UK Ltd),left_only
7,amp,3294611000001104,0302000K0BDABAM,Symbicort 200/6 Turbohaler (AstraZeneca UK Ltd),left_only
8,amp,4373811000001100,0302000K0BDACAU,Symbicort 400/12 Turbohaler (AstraZeneca UK Ltd),both
9,amp,32926011000001100,0302000K0BDADBB,Symbicort 200micrograms/dose / 6micrograms/dose pressurised inhaler (AstraZeneca UK Ltd),left_only


## Low and medium dose Combination ICS <a id='lm'></a>

In [20]:
#ones that = left only are low medium dose
low_med_ics_combi_agent = combine.loc[(combine['_merge'] == "left_only")]
low_med_ics_combi_agent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 68
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   type      43 non-null     object  
 1   id        43 non-null     int64   
 2   bnf_code  43 non-null     object  
 3   nm        43 non-null     object  
 4   _merge    43 non-null     category
dtypes: category(1), int64(1), object(3)
memory usage: 1.8+ KB


In [21]:
low_med_ics_combi_agent.sort_values(["type", "nm"], inplace=True)
low_med_ics_combi_agent.drop('_merge', 1, inplace=True)

<ipython-input-21-a591b03cbe97>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_med_ics_combi_agent.sort_values(["type", "nm"], inplace=True)
/root/.pyenv/versions/3.8.1/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [22]:
low_med_ics_combi_agent

,type,id,bnf_code,nm
30,amp,34215311000001107,0302000N0BGABBF,AirFluSal 25micrograms/dose / 125micrograms/dose inhaler (Sandoz Ltd)
35,amp,34677011000001107,0302000N0BJAABF,Aloflute 25micrograms/dose / 125micrograms/dose inhaler (Mylan)
40,amp,35594211000001100,0302000N0BLABBF,Combisal 25micrograms/dose / 125micrograms/dose inhaler (Aspire Pharma Ltd)
39,amp,35594011000001105,0302000N0BLAABE,Combisal 25micrograms/dose / 50micrograms/dose inhaler (Aspire Pharma Ltd)
10,amp,25254111000001105,0302000K0BHAAAM,DuoResp Spiromax 160micrograms/dose / 4.5micrograms/dose dry powder inhaler (Teva UK Ltd)
22,amp,21019411000001101,0302000N0BDAABJ,Flutiform 125micrograms/dose / 5micrograms/dose inhaler (Napp Pharmaceuticals Ltd)
24,amp,21020611000001104,0302000N0BDACBL,Flutiform 50micrograms/dose / 5micrograms/dose inhaler (Napp Pharmaceuticals Ltd)
25,amp,35647311000001101,0302000N0BDADBP,Flutiform K-haler 125micrograms/dose / 5micrograms/dose breath actuated inhaler (Napp Pharmaceuticals Ltd)
26,amp,35650811000001109,0302000N0BDAEBQ,Flutiform K-haler 50micrograms/dose / 5micrograms/dose breath actuated inhaler (Napp Pharmaceuticals Ltd)
13,amp,34950311000001108,0302000K0BIABAM,Fobumix Easyhaler 160micrograms/dose / 4.5micrograms/dose dry powder inhaler (Orion Pharma (UK) Ltd)


In [25]:
low_med_ics_combi_agent.to_csv(os.path.join('..','data','low_med_inhaledsteroid_combiagent_codelist.csv'))